# Leonardo's Self-Propelled Cart: The First Robot

> *"I have discovered that a screw of such a nature as to be able to move a weight..."*  
> — Leonardo da Vinci

## Introduction
Long before the automobile, Leonardo designed a self-propelled cart powered by coiled springs. It featured a complex **escapement mechanism** to regulate speed and even programmable steering.

In this notebook, we analyze the powertrain:
1.  **Spring Energy**: Storing potential energy in leaf springs.
2.  **Gear Reduction**: Converting high torque to speed.
3.  **Escapement**: Regulating the release of energy.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Spring Potential Energy
The energy stored in a torsion spring is:
$$ E = \frac{1}{2} k \theta^2 $$
Where $k$ is the spring constant and $\theta$ is the winding angle.

### 2. Vehicle Dynamics
The cart accelerates based on the torque delivered to the wheels:
$$ F_{drive} = \frac{\tau_{spring} \cdot N_{gear}}{r_{wheel}} $$
$$ m a = F_{drive} - F_{drag} - F_{friction} $$

### 3. Escapement
Without an escapement, the spring would unwind instantly. The escapement limits the maximum rotational speed $\omega_{max}$.

In [ ]:
def simulate_cart(spring_turns, gear_ratio, mass_kg):
    dt = 0.05
    time = np.arange(0, 30, dt)
    
    # Parameters
    k_spring = 50.0 # Nm/rad
    r_wheel = 0.15 # m
    drag_coeff = 0.8
    area = 0.5
    rho = 1.225
    friction = 2.0 # N
    
    # Initial State
    theta_spring = spring_turns * 2 * np.pi
    velocity = 0.0
    position = 0.0
    
    # History
    pos_hist = []
    vel_hist = []
    energy_hist = []
    
    for t in time:
        # Spring Torque
        torque = k_spring * theta_spring if theta_spring > 0 else 0
        
        # Drive Force
        drive_force = (torque * gear_ratio) / r_wheel
        
        # Escapement Limit (Simplified)
        # Limits max power output
        max_power = 100.0 # Watts
        if drive_force * velocity > max_power:
            drive_force = max_power / velocity
            
        # Resistance
        drag = 0.5 * rho * drag_coeff * area * velocity**2
        net_force = drive_force - drag - friction
        
        # Dynamics
        accel = net_force / mass_kg
        velocity += accel * dt
        if velocity < 0: velocity = 0
        
        position += velocity * dt
        
        # Unwind Spring
        # Wheel rotation -> Spring rotation
        d_theta_wheel = (velocity * dt) / r_wheel
        d_theta_spring = d_theta_wheel / gear_ratio
        theta_spring -= d_theta_spring
        
        pos_hist.append(position)
        vel_hist.append(velocity)
        energy_hist.append(0.5 * k_spring * max(0, theta_spring)**2)
        
    return time, pos_hist, vel_hist, energy_hist

def plot_cart(turns, ratio, mass):
    t, pos, vel, energy = simulate_cart(turns, ratio, mass)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Motion
    ax1.plot(t, vel, 'k-', linewidth=2, label='Velocity (m/s)')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(t, pos, 'b--', label='Distance (m)')
    
    ax1.set_title('Cart Motion')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Velocity (m/s)')
    ax1_twin.set_ylabel('Distance (m)', color='b')
    
    # Plot 2: Energy
    ax2.plot(t, energy, 'g-', linewidth=2, label='Spring Energy')
    ax2.set_title('Potential Energy Depletion')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Energy (J)')
    ax2.fill_between(t, 0, energy, color='green', alpha=0.1)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Total Distance: {pos[-1]:.1f} m")
    print(f"Max Speed: {max(vel):.1f} m/s")

interact(plot_cart, 
         turns=widgets.IntSlider(min=5, max=50, step=5, value=20, description='Spring Turns'),
         ratio=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.2, description='Gear Ratio'),
         mass=widgets.IntSlider(min=20, max=100, step=5, value=50, description='Mass (kg)'));

## Conclusion

The simulation reveals the trade-off in gear ratios: a low gear ratio provides high torque for acceleration but unwinds the spring quickly (short range). A high gear ratio extends the range but limits acceleration.

Leonardo's inclusion of an **escapement** was brilliant—it effectively governed the power release, allowing the cart to travel at a consistent speed for a theatrical performance rather than just sprinting and stopping.